<a href="https://colab.research.google.com/github/girotodenis/python/blob/master/relatorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
data_i = datetime(2020, 1, 11)
data_f = datetime(2020, 5, 17)


import numpy as np
import pandas as pd
from datetime import datetime
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:

#!git clone -l -s git://github.com/girotodenis/data_science.git cloned-repo
#%cd cloned-repo
#!ls
df = pd.read_excel('/content/ePol.xlsx')
#print(df.columns)
#print(df['Progresso'].unique())
df = df[(df['Progresso']!='Não iniciada')]

df['Data de início'].fillna(df['Criado em'], inplace=True)
df['Data de conclusão'].fillna(df['Concluído em'], inplace=True)
df['Data de início'] = pd.to_datetime(df['Data de início'])
df['Data de conclusão'] = pd.to_datetime(df['Data de conclusão'])
df['Criado em'] = pd.to_datetime(df['Criado em'])
df['Concluído em'] = pd.to_datetime(df['Concluído em'])

df['Atribuído a'].fillna(df['Criado por'], inplace=True)
#df['Concluído em'].fillna(df['Criado por'], inplace=True)

df = df[['Nome da tarefa','Nome do Bucket','Progresso','Atribuído a', 'Criado por', 'Criado em',
       'Data de início', 'Data de conclusão']]

data_minima = df['Data de início'].min()
data_maxima = df['Data de conclusão'].max()


In [0]:
sequencia_dias = pd.DataFrame(data={'Dia':[]})
sequencia_dias['Dia'] = pd.date_range(start=data_minima,end=data_maxima, name="dia")
df_completo = pd.DataFrame(columns=df.columns)
df_completo['Dia'] = None

for data_corrente in sequencia_dias['Dia'].values: 
  x = df[ ((df['Data de início']<=data_corrente) & (df['Progresso']!='Concluída')) | ((df['Data de início']<=data_corrente) & (df['Data de conclusão']>=data_corrente) ) ]
  x['Dia'] = data_corrente
  x['count'] = 1
  df_completo = df_completo.append(x, ignore_index=True) 

#print(df.columns)
#print(df['Atribuído a'].unique())

nome = 'Denis Silva Giroto'


In [0]:
qtd_demanda = df_completo[['Dia', 'Atribuído a', 'count']]
qtd_demanda = qtd_demanda.groupby(['Dia', 'Atribuído a']).count()
qtd_demanda = qtd_demanda.reset_index()
qtd_demanda['Dia'] = pd.to_datetime(qtd_demanda['Dia'])


In [0]:
df_x = qtd_demanda[ ((qtd_demanda['Dia']>=data_i) & (qtd_demanda['Dia']<=data_f) ) ]
df_x['Dia'] = df_x['Dia'].astype(str)

#xs = df_x.pivot(index='Dia', columns='Atribuído a', values='count')
#xs.plot(figsize = (15,4))

In [0]:
fig, ax = plt.subplots(figsize = (20,8))

values = ['g','b','r','y']
i=0
for nome in df_x['Atribuído a'].unique():
  color = values[i]
  filtro = df_x[(df_x['Atribuído a']==nome)]
  x = filtro['Dia'] #Setar a variável do eixo X
  y = filtro['count'] #Setar a variável do eixo Y
  ax.plot(x, y, 'k--', label = nome, color=color,  marker = 'o') # Criar a linha
  i+=1

plt.xticks(rotation = 90) #Fazer a Rotação do eixo x
plt.grid() #Aplicar a linha grade
plt.tight_layout() #Ajustar o layout
plt.legend(loc = 1) #Plotar Legenda
plt.title('Gráfico de Controle', size = 30, weight = 'bold',
          color = '#006400', style = 'italic') 
plt.xlabel('Dias', size = 10)
plt.ylabel('Qt de tarefas', size = 10)
plt.show()